# Analysis of Bangkok's Airbnb Listings

<img src="./img/airbnb_bangkok.png" alt="drawing" style="width:90%;"/>

## Introduction
---

The rapid growth of Airbnb has revolutionized the way travelers find accommodations, offering many different rental options in cities around the world. This data analysis project focuses on understanding the dynamics of Airbnb listings in Bangkok, a vibrant city known for its cultural heritage, tourist attractions and amazing cuisine. By analyzing various aspects of Airbnb listings, such as pricing, room type, location, and guest reviews, this project is meant to give a deeper understanding of the Airbnb listings in Bangkok and provide insights that can benefit hosts and guests using Airbnb.

**Objectives:**

- **Pricing Analysis**: Examine the distribution of nightly rates and identify factors influencing pricing.
- **Room Type Analysis**: Examine the distribution of different room types (entire home/apt, private room, shared room) and their impact on pricing
- **Geographical Distribution**: Map listings to visualize popular areas and examine neighborhood-level trends.
- **Review Analysis**: Investigate listing popularity based on the number of reviews

**Dataset:**
This project uses [InsideAirbnb](http://insideairbnb.com) listings data for Bangkok as of December 2023

#### Data Dictionary

| Column                            | Explanation                                                                                                                                                                                        |
| --------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| id                                | Airbnb's unique identifier for the listing                                                                                                                                                         |
| name                              |                                                                                                                                                                                                    |
| host\_id                          |                                                                                                                                                                                                    |
| host\_name                        |                                                                                                                                                                                                    |
| neighbourhood\_group              | The neighbourhood group as geocoded using the latitude and longitude against neighborhoods as defined by open or public digital shapefiles.                                                        |
| neighbourhood                     | The neighbourhood as geocoded using the latitude and longitude against neighborhoods as defined by open or public digital shapefiles.                                                              |
| latitude                          | Uses the World Geodetic System (WGS84) projection for latitude and longitude.                                                                                                                      |
| longitude                         | Uses the World Geodetic System (WGS84) projection for latitude and longitude.                                                                                                                      |
| room\_type                        |                                                                                                                                                                                                    |
| price                             | daily price in local currency. Note, $ sign may be used despite locale                                                                                                                             |
| minimum\_nights                   | minimum number of night stay for the listing (calendar rules may be different)                                                                                                                     |
| number\_of\_reviews               | The number of reviews the listing has                                                                                                                                                              |
| last\_review                      | The date of the last/newest review                                                                                                                                                                 |
| calculated\_host\_listings\_count | The number of listings the host has in the current scrape, in the city/region geography.                                                                                                           |
| availability\_365                 | avaliability\_x. The availability of the listing x days in the future as determined by the calendar. Note a listing may be available because it has been booked by a guest or blocked by the host. |
| number\_of\_reviews\_ltm          | The number of reviews the listing has (in the last 12 months)                                                                                                                                      |
| license                           |                                                                                                                                                                                                    |
[Source](http://insideairbnb.com/get-the-data.html) and [license](https://creativecommons.org/licenses/by/4.0/) of dataset. 

## Data Wrangling and Cleaning
---

### Importing the data

In [1]:
import pandas as pd
import numpy as np

# Loading in data
listing = pd.read_csv("data/listings.csv", index_col=0)
listing.info()
listing.head()

<class 'pandas.core.frame.DataFrame'>
Index: 22104 entries, 27934 to 1053584060635211891
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            22104 non-null  object 
 1   host_id                         22104 non-null  int64  
 2   host_name                       22103 non-null  object 
 3   neighbourhood_group             0 non-null      float64
 4   neighbourhood                   22104 non-null  object 
 5   latitude                        22104 non-null  float64
 6   longitude                       22104 non-null  float64
 7   room_type                       22104 non-null  object 
 8   price                           21582 non-null  float64
 9   minimum_nights                  22104 non-null  int64  
 10  number_of_reviews               22104 non-null  int64  
 11  last_review                     14162 non-null  object 
 12  reviews_per_month  

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
id,,,,,,,,,,,,,,,,,
27934,Condo in Samsen Nai · ★4.85 · 1 bedroom · 1 be...,120437,Nuttee,NaN,Ratchathewi,13.75983,100.54134,Entire home/apt,1903.0,3,64,2020-01-06,0.45,2,329,0,NaN
27979,Rental unit in Bangkok · 1 bedroom · 2 beds · ...,120541,Emy,NaN,Bang Na,13.66818,100.61674,Private room,1316.0,1,0,NaN,NaN,2,0,0,NaN
28745,Rental unit in Bangkok · 1 bedroom · 1 bed · 1...,123784,Familyroom,NaN,Bang Kapi,13.75232,100.62402,Private room,800.0,60,0,NaN,NaN,1,0,0,NaN
820395,Guesthouse in Bangkok · ★4.77 · 1 bedroom · 1 ...,822284,Rae And Charlie,NaN,Bang Sue,13.83026,100.52082,Entire home/apt,700.0,7,13,2018-02-14,0.10,8,349,0,NaN
35780,Rental unit in Bangkok · ★4.75 · 1 bedroom · 1...,153730,Sirilak,NaN,Din Daeng,13.78823,100.57256,Private room,1286.0,14,4,2023-04-03,0.04,1,327,2,NaN


### Cleaning the data

In [2]:
# Replacing NaN values with 0
values = {"price": 0, "reviews_per_month": 0}
listing.fillna(value = values, inplace=True)
listing.head()

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
id,,,,,,,,,,,,,,,,,
27934,Condo in Samsen Nai · ★4.85 · 1 bedroom · 1 be...,120437,Nuttee,NaN,Ratchathewi,13.75983,100.54134,Entire home/apt,1903.0,3,64,2020-01-06,0.45,2,329,0,NaN
27979,Rental unit in Bangkok · 1 bedroom · 2 beds · ...,120541,Emy,NaN,Bang Na,13.66818,100.61674,Private room,1316.0,1,0,NaN,0.00,2,0,0,NaN
28745,Rental unit in Bangkok · 1 bedroom · 1 bed · 1...,123784,Familyroom,NaN,Bang Kapi,13.75232,100.62402,Private room,800.0,60,0,NaN,0.00,1,0,0,NaN
820395,Guesthouse in Bangkok · ★4.77 · 1 bedroom · 1 ...,822284,Rae And Charlie,NaN,Bang Sue,13.83026,100.52082,Entire home/apt,700.0,7,13,2018-02-14,0.10,8,349,0,NaN
35780,Rental unit in Bangkok · ★4.75 · 1 bedroom · 1...,153730,Sirilak,NaN,Din Daeng,13.78823,100.57256,Private room,1286.0,14,4,2023-04-03,0.04,1,327,2,NaN


In [3]:
# Updating dataset by filtering out free listings
listing = listing[listing.price > 0]
listing.info()
listing.head()

<class 'pandas.core.frame.DataFrame'>
Index: 21582 entries, 27934 to 1053584060635211891
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            21582 non-null  object 
 1   host_id                         21582 non-null  int64  
 2   host_name                       21581 non-null  object 
 3   neighbourhood_group             0 non-null      float64
 4   neighbourhood                   21582 non-null  object 
 5   latitude                        21582 non-null  float64
 6   longitude                       21582 non-null  float64
 7   room_type                       21582 non-null  object 
 8   price                           21582 non-null  float64
 9   minimum_nights                  21582 non-null  int64  
 10  number_of_reviews               21582 non-null  int64  
 11  last_review                     13852 non-null  object 
 12  reviews_per_month  

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
id,,,,,,,,,,,,,,,,,
27934,Condo in Samsen Nai · ★4.85 · 1 bedroom · 1 be...,120437,Nuttee,NaN,Ratchathewi,13.75983,100.54134,Entire home/apt,1903.0,3,64,2020-01-06,0.45,2,329,0,NaN
27979,Rental unit in Bangkok · 1 bedroom · 2 beds · ...,120541,Emy,NaN,Bang Na,13.66818,100.61674,Private room,1316.0,1,0,NaN,0.00,2,0,0,NaN
28745,Rental unit in Bangkok · 1 bedroom · 1 bed · 1...,123784,Familyroom,NaN,Bang Kapi,13.75232,100.62402,Private room,800.0,60,0,NaN,0.00,1,0,0,NaN
820395,Guesthouse in Bangkok · ★4.77 · 1 bedroom · 1 ...,822284,Rae And Charlie,NaN,Bang Sue,13.83026,100.52082,Entire home/apt,700.0,7,13,2018-02-14,0.10,8,349,0,NaN
35780,Rental unit in Bangkok · ★4.75 · 1 bedroom · 1...,153730,Sirilak,NaN,Din Daeng,13.78823,100.57256,Private room,1286.0,14,4,2023-04-03,0.04,1,327,2,NaN


After removing all the free listings in our dataset, we can see that the total number of entries went from 22104 to 21582, which means there are 522 listings that are free.